I have took a few references from my homework 4 code

In [ ]:
! pip install datasets transformers

In [ ]:
model_checkpoint = "roberta-base"
batch_size = 16

In [ ]:
from datasets import load_dataset

dataset = load_dataset("raquiba/Sarcasm_News_Headline")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['is_sarcastic', 'headline', 'article_link'],
        num_rows: 28619
    })
    test: Dataset({
        features: ['is_sarcastic', 'headline', 'article_link'],
        num_rows: 26709
    })
})

In [ ]:
dataset["train"] = dataset["train"].remove_columns("article_link")
dataset["test"] = dataset["test"].remove_columns("article_link")

In [ ]:
dataset["train"] = dataset["train"].rename_column("is_sarcastic", "label")
dataset["test"] = dataset["test"].rename_column("is_sarcastic", "label")

In [ ]:
from datasets import DatasetDict
test_valid = dataset["test"].train_test_split(test_size=0.7)

train_test_valid_dataset = DatasetDict({
    'train': dataset['train'],
    'test': test_valid['test'],
    'validation': test_valid['train']})

In [ ]:
train_test_valid_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'headline'],
        num_rows: 28619
    })
    test: Dataset({
        features: ['label', 'headline'],
        num_rows: 18697
    })
    validation: Dataset({
        features: ['label', 'headline'],
        num_rows: 8012
    })
})

In [ ]:
train_test_valid_dataset["test"][0]

{'label': 0,
 'headline': 'finding kind: a film and campaign that asks girls to be a little kinder'}

In [ ]:
# As is
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
# As is
show_random_elements(train_test_valid_dataset["test"])

,label,headline
0,1,evil hong kong kung-fu legions petition for right to attack two at a time
1,0,who says all countries should tax sugary drinks to curb obesity
2,1,clinton reminds new yorkers she moved there hoping career dreams would work out too
3,0,here's a pro tip for katie couric before she does another documentary
4,0,"new web series aims to tell stories of love, addiction and healing"
5,1,wife unfazed by husband's sad e-mails to other women
6,0,photography series spotlighting iconic women over 70 proves the best is yet to come
7,0,"after learning about homelessness, kind toddler starts donation drive"
8,1,bartender hurt by unfinished drink
9,1,ryan chugs down rhino horn and bull semen shake for mid-debate boost


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# As is
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
#As is
def preprocess_function(examples):
    return tokenizer(examples['headline'], truncation=True)

In [ ]:
#As is
preprocess_function(train_test_valid_dataset['train'][:5])

{'input_ids': [[0, 212, 32430, 18891, 4211, 18834, 109, 41334, 6700, 9, 2549, 872, 2], [0, 12789, 2851, 4, 4940, 20941, 596, 12442, 16, 3064, 765, 15, 3959, 6, 6689, 9057, 2], [0, 21046, 110, 32065, 35, 361, 24234, 9997, 430, 13204, 2], [0, 3976, 13767, 1650, 17410, 28587, 31, 562, 7, 173, 2], [0, 19456, 606, 1256, 593, 7, 634, 2136, 128, 8656, 154, 108, 12461, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
encoded_dataset = train_test_valid_dataset.map(preprocess_function, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

In [ ]:
model_name = model_checkpoint.split("/")[-1]

#modified arguments parameter
args = TrainingArguments(
    f"{model_name}-finetuned-sarcasm-news-headline-detection",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    # load_best_model_at_end=True,
    # metric_for_best_model=metric_name,
    push_to_hub=True,
    overwrite_output_dir=True,
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer
)

In [ ]:
trainer.train()
trainer.save_model()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,0.232500,0.123460
2,0.152500,0.076740
3,0.094400,0.045069


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 1.00/476M [00:00<?, ?B/s]

Upload file runs/Apr08_21-56-44_280c3311dc1e/events.out.tfevents.1680991043.280c3311dc1e.207.0:   0%|         …

To https://huggingface.co/sadia72/roberta-base-finetuned-sarcasm-news-headline-detection
   efc087c..3c301a9  main -> main

   efc087c..3c301a9  main -> main

To https://huggingface.co/sadia72/roberta-base-finetuned-sarcasm-news-headline-detection
   3c301a9..06d0c4f  main -> main

   3c301a9..06d0c4f  main -> main



In [ ]:
!pip install evaluate

In [ ]:
predictions = trainer.predict(encoded_dataset["test"])
#print(predictions)
print(len(predictions.predictions), len(predictions.label_ids))

18697 18697


In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
import evaluate

metric = evaluate.load("accuracy")
metric.compute(references=predictions.label_ids, predictions=preds,)

{'accuracy': 0.9871637161041878}

In [ ]:
import evaluate

f1_metric = evaluate.load("f1")
f1_metric.compute(references=predictions.label_ids, predictions=preds,)

{'f1': 0.9853049228508449}

In [ ]:
trainer.evaluate()

{'eval_loss': 0.04506874829530716,
 'eval_runtime': 22.2999,
 'eval_samples_per_second': 359.284,
 'eval_steps_per_second': 44.933,
 'epoch': 3.0}

In [ ]:
from transformers import pipeline

classifier = pipeline('text-classification',model='./roberta-base-finetuned-sarcasm-news-headline-detection', tokenizer='roberta-base')

In [ ]:
classifier('I clapped because it’s finished, not because I like it.')

[{'label': 'LABEL_0', 'score': 0.9997033476829529}]

In [ ]:
classifier('If you think you are too small to be effective, you have never been in the dark with a mosquito.')

[{'label': 'LABEL_0', 'score': 0.9995872378349304}]

In [ ]:
classifier('If had a dollar for every smart thing you say. I’ll be poor.')

[{'label': 'LABEL_0', 'score': 0.9995858073234558}]

In [ ]:
classifier("hillary clinton spends busy day fueling speculation, not ruling things out.")

[{'label': 'LABEL_1', 'score': 0.8853675723075867}]

In [ ]:
classifier("bush arrives at debate wearing flight suit")

[{'label': 'LABEL_1', 'score': 0.9997857213020325}]

In [ ]:
classifier("Me pretending to listen should be enough for you.")

[{'label': 'LABEL_0', 'score': 0.9996789693832397}]

In [ ]:
classifier("bartender hurt by unfinished drink.")

[{'label': 'LABEL_1', 'score': 0.9998310804367065}]

In [ ]:
classifier("who says all countries should tax sugary drinks to curb obesity")

[{'label': 'LABEL_0', 'score': 0.9997809529304504}]

In [ ]:
classifier("California court rules bees are now fish")

[{'label': 'LABEL_1', 'score': 0.9413052201271057}]